In [1]:
import datetime
import random
import time

import baostock as bs
import pandas as pd

from tqdm import tqdm

In [ ]:
start_date = '1999-01-01'
end_date = datetime.date.today().isoformat()

def print_error(rs):
    if rs.error_code != '0':
        print('respond error_code:'+rs.error_code)
        print('respond  error_msg:'+rs.error_msg)

        
bs.login()

rs = bs.query_trade_dates(start_date=start_date, end_date=end_date)
print_error(rs)
df = rs.get_data()
trading_day = df[df['is_trading_day'] == '1']['calendar_date']
trading_day = trading_day[0::11]

data_df = pd.DataFrame()
for date in tqdm(trading_day):
    stock_rs = bs.query_all_stock(date)
    print_error(stock_rs)        
    stock_df = stock_rs.get_data()
    stock_df = stock_df[stock_df['code_name'].str.contains('指数') == False]
    
    for code in stock_df["code"]:        
        k_rs = bs.query_history_k_data_plus(code, "date,code,peTTM", date, date)
        print_error(k_rs)
        data_df = data_df.append(k_rs.get_data())
    
    data_df.to_csv("peTTM.csv", index=False)
    time.sleep(random.random())

bs.logout()

  0%|          | 0/496 [00:00<?, ?it/s]

login success!


 41%|████▏     | 205/496 [4:44:13<9:41:13, 119.84s/it] 

In [53]:
rs = bs.query_history_k_data_plus("sh.600320", "date,code,peTTM", date, date)
df = df.append(rs.get_data(), ignore_index=True)

In [100]:
code

'sz.000405'

In [57]:
rs = bs.query_trade_dates(start_date="1999-01-01", end_date="2021-07-04")

In [58]:
rs.error_code

'0'

In [80]:
df = bs.query_all_stock(date).get_data()

In [63]:
date.today().isoformat()

'2021-07-04'

In [61]:
from datetime import date

In [88]:
df[df['code_name'].str.contains('指数') == False]

,code,tradeStatus,code_name
8,sh.600001,1,邯郸钢铁
9,sh.600002,1,齐鲁石化
10,sh.600009,1,上海机场
11,sh.600051,1,宁波联合
12,sh.600052,1,浙江广厦
...,...,...,...
832,sz.000895,1,双汇发展
833,sz.000898,1,鞍钢股份
834,sz.000899,1,赣能股份
835,sz.001696,1,宗申动力


In [89]:
df

,code,tradeStatus,code_name
0,sh.000001,1,上证综合指数
1,sh.000002,1,上证A股指数
2,sh.000003,1,上证B股指数
3,sh.000004,1,上证工业类指数
4,sh.000005,1,上证商业类指数
...,...,...,...
838,sz.399002,1,深证成份指数(收益)
839,sz.399003,1,深证成份B股指数
840,sz.399106,1,深证综合指数
841,sz.399107,1,深证A股指数


In [92]:
bs.login()

rs = bs.query_trade_dates(start_date=start_date, end_date=end_date)
print_error(rs)
df = rs.get_data()
trading_day = df[df['is_trading_day'] == '1']['calendar_date']

login success!


In [95]:
trading_day[0::11]

3       1999-01-04
18      1999-01-19
33      1999-02-03
67      1999-03-09
82      1999-03-24
           ...    
8148    2021-04-23
8168    2021-05-13
8183    2021-05-28
8201    2021-06-15
8216    2021-06-30
Name: calendar_date, Length: 496, dtype: object